# Arsing about with mutable and immutable things

Some quick tests of speed and ways that one might have split structs as a way to track objects which have a limited mutability

In [1]:
using BenchmarkTools
using Accessors

In [2]:
struct imu 
    m::Int
    n::Int
    f::Float64
    g::Float64
end


In [3]:
mutable struct mu
    m::Int
    n::Int
    f::Float64
    g::Float64
end

In [12]:
@benchmark begin
    vimu = Vector{imu}()
    sizehint!(vimu, 10000)
    for i in 1:10000
        push!(vimu, imu(i, i, i*π, 7.6*i))
    end
    for i in 1:10000
        @reset vimu[i].m += 100
    end
end

BenchmarkTools.Trial: 19 samples with 1 evaluation.
 Range (min … max):  260.480 ms … 275.679 ms  ┊ GC (min … max): 20.57% … 20.89%
 Time  (median):     266.475 ms               ┊ GC (median):    20.35%
 Time  (mean ± σ):   266.767 ms ±   4.277 ms  ┊ GC (mean ± σ):  20.31% ±  0.99%

  ▁  ▁    █▁▁         ▁  ██  ▁ ▁▁ ▁ ▁             ▁         ▁ ▁  
  █▁▁█▁▁▁▁███▁▁▁▁▁▁▁▁▁█▁▁██▁▁█▁██▁█▁█▁▁▁▁▁▁▁▁▁▁▁▁▁█▁▁▁▁▁▁▁▁▁█▁█ ▁
  260 ms           Histogram: frequency by time          276 ms <

 Memory estimate: 2.98 GiB, allocs estimate: 20002.

In [13]:
@benchmark begin
    vmu = Vector{mu}()
    sizehint!(vmu, 10000)
    for i in 1:10000
        push!(vmu, mu(i, i, i*π, 7.6*i))
    end
    for i in 1:10000
        vmu[i].m += 100
    end
end

BenchmarkTools.Trial: 10000 samples with 1 evaluation.
 Range (min … max):  74.291 μs …  1.587 ms  ┊ GC (min … max):  0.00% … 93.11%
 Time  (median):     81.459 μs              ┊ GC (median):     0.00%
 Time  (mean ± σ):   92.698 μs ± 98.163 μs  ┊ GC (mean ± σ):  11.23% ±  9.82%

  █▅                                                          ▁
  ████▄▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▃▅▅▆ █
  74.3 μs      Histogram: log(frequency) by time       818 μs <

 Memory estimate: 546.98 KiB, allocs estimate: 10002.

## Experiment with a stuct that has a mutable subcomponent

In [14]:
struct mix
    i::imu
    m::mu
end

In [16]:
m = mix(imu(2, 3, 5.7, 8.0), mu(3, 6, 1.2, 9.0))

mix(imu(2, 3, 5.7, 8.0), mu(3, 6, 1.2, 9.0))

In [18]:
m.i.f

5.7

In [19]:
m.i.f = π

ErrorException: setfield!: immutable struct of type imu cannot be changed

In [20]:
m.m.f

1.2

In [21]:
m.m.f = π

π = 3.1415926535897...

**It works!**

In [22]:
m

mix(imu(2, 3, 5.7, 8.0), mu(3, 6, 3.141592653589793, 9.0))